In [3]:
%matplotlib inline
import pandas as pd
import os
import random
import glob
import torch
import torchaudio
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display, Audio

import librosa
import seaborn as sns
import shutil
import collections
import contextlib
import sys
import wave
import webrtcvad
import librosa
import soundfile


import concurrent.futures
from wvmos import get_wvmos
from pydub import AudioSegment

from denoiser import pretrained
from denoiser.dsp import convert_audio

In [4]:
females = pd.read_excel('females.xlsx')

In [5]:
client_ids = females.client_id.value_counts().iloc[:20].index.tolist()
twenty_females= females[females.client_id.isin(client_ids)]

In [6]:
twenty_females.sort_values(by=['path'], inplace=True)

/tmp/ipykernel_9797/2113391793.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twenty_females.sort_values(by=['path'], inplace=True)


In [7]:
six_females = twenty_females[
    (females.client_id == '77844f4f66a5cd12f1aa1dd9f236547e9d8dc724641c17aeba50b2dd7ae7b457af95d55838392288c6ed2b7881a617c7bf3f51edda9dfe8c3a882c0669baa1c7') |
    (females.client_id == '66858b5ce4429ee1297e082789724acb02c7c3d38e543b03815604906fa731232f78b0a5b822b036f3d856b27f3815e1837349b842f8aa6e0be7ab68940e39be') |
    (females.client_id == 'a59598f0a310679014e8663eeed32c7408fa0b0922e9e3f4eb4a759eb199342345753267b9c0cc4116d838f73c7f4b49350b4bd646caee4ac50935b60186d9f3') |
    (females.client_id == '6e09ff9cc1f149df26e8db256a211d367bced4288641e22c9dd82fbae9ad823f49058c132ab42e9d942ed3d3e24ac64b96bbe7419bb43d4a40c3ba3c1c7223b2') |
    (females.client_id == 'dec0c7bd1c3b6b23e33c8a289e9b37541ae9ec7a57697a72f275ee010799d658ed122c00988fd1f29baa0cca9425962bac63585808549d3fa638fb1fb64b2a96') |
    (females.client_id == 'a1313c5f1f30851ca0eb76c4ce9080b2ddc359b0111d1e8a9de1cd37f15e954b3bff006047b54e726ca7f8c78c7b8527a3e62d47c9f8b71f869a9f4bbb1dbbf1')

]

/tmp/ipykernel_9797/1542461241.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  six_females = twenty_females[


In [8]:
six_females['duration_seconds'] = six_females['duration[ms]']/1000

/tmp/ipykernel_9797/4044183189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  six_females['duration_seconds'] = six_females['duration[ms]']/1000


In [9]:
six_females['path'] = six_females['path'].apply(lambda x: x.replace('.mp3', '.wav'))

/tmp/ipykernel_9797/3817877466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  six_females['path'] = six_females['path'].apply(lambda x: x.replace('.mp3', '.wav'))


In [19]:
# !mkdir 'six_females_wavs'

In [21]:
for file_name in tqdm(six_females.path):
    source = 'female_wavs/' + file_name
    destination = 'six_females_wavs/' + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)

  0%|          | 0/18028 [00:00<?, ?it/s]

## Voice Activity Detection

In [10]:
def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        sample_rate = 32000  # We will pretend it's sampled at 16k
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.5 * ring_buffer.maxlen:
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []

    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

In [11]:
wav_32k_dir = 'six_females_wavs/'
filtered_tmp_dir = 'filtered_six_wavs/'
final_wav_dir = 'new_six_wavs/'
SAMPLE_RATE = 22050

!mkdir 'filtered_six_wavs/'
!mkdir 'new_six_wavs/'

bad_files = []

mkdir: cannot create directory ‘filtered_six_wavs/’: File exists
mkdir: cannot create directory ‘new_six_wavs/’: File exists


In [12]:
def clip_wav_to_speech_and_save(wav_path):
    filename = os.path.basename(wav_path)
    if not os.path.exists(os.path.join(final_wav_dir, filename)):
        audio, sample_rate = read_wave(
            os.path.join(wav_32k_dir, filename))
        vad = webrtcvad.Vad(3)
        frames = frame_generator(30, audio, sample_rate)
        frames = list(frames)
        segments = vad_collector(sample_rate, 30, 300, vad, frames)
        speech = b''.join(list(segments))
        # Likely to be a problem if less than 1s of speech
        if len(speech) < sample_rate * 2:
            bad_files.append(wav_path)        
        else:
            write_wave(
                os.path.join(filtered_tmp_dir, filename),
                speech, sample_rate)
            # Resample to SAMPLE_RATE
            wav, _ = librosa.load(
                os.path.join(filtered_tmp_dir, filename),
                sr=SAMPLE_RATE)
            soundfile.write(
                os.path.join(final_wav_dir, filename),
                wav, SAMPLE_RATE)  # Trim a little extra from the end  
for wav_path in tqdm(six_females.path):
    clip_wav_to_speech_and_save(wav_path)

  0%|          | 0/18028 [00:00<?, ?it/s]

In [13]:
bad_files

['common_voice_sw_30129692.wav',
 'common_voice_sw_31268635.wav',
 'common_voice_sw_31268795.wav',
 'common_voice_sw_31270475.wav']

In [14]:
six_females = six_females[~six_females.path.isin(bad_files)]

print(f'Remove {len(bad_files)} files without speech, new total: {len(six_females)}.')

Remove 4 files without speech, new total: 18024.


In [15]:
six_females.to_csv('new_six.csv', index=False)

In [16]:
six_females.sort_values(by=['path'], inplace=True)

## Denoiser

In [17]:
!mkdir "new_six_denoised/"

mkdir: cannot create directory ‘new_six_denoised/’: File exists


In [18]:
noisy_audio_dir = "new_six_wavs/"
denoised_audio_dir = "new_six_denoised/"

In [19]:
# Load denoising model
model = pretrained.dns64().cuda()

# Loop through each audio file in the noisy audio directory
for audio_file in tqdm(six_females.path):
    # Load noisy audio
    noisy_audio, sr = torchaudio.load(os.path.join(noisy_audio_dir, audio_file))

    # Denoise audio
    denoised_audio = convert_audio(noisy_audio.cuda(), sr, model.sample_rate, model.chin)
    with torch.no_grad():
        denoised_audio = model(denoised_audio[None])[0]

    # Save denoised audio
    denoised_audio_path = os.path.join(denoised_audio_dir, audio_file)
    torchaudio.save(denoised_audio_path, denoised_audio.cpu(), model.sample_rate)

  0%|          | 0/18024 [00:00<?, ?it/s]

In [20]:
# Set the source directory
src_dir = "new_six_denoised/"

# Set the target sample rate
target_sr = 22050

for filename in tqdm(os.listdir(src_dir)):
    # Check if the file is a .wav file
    if filename.endswith(".wav"):
        # Load the audio file
        filepath = os.path.join(src_dir, filename)
        audio = AudioSegment.from_wav(filepath)

        # Change the sample rate and save back to the same location
        audio = audio.set_frame_rate(target_sr)
        audio.export(filepath, format="wav")

  0%|          | 0/18024 [00:00<?, ?it/s]

## WV-MOS

In [21]:
model = get_wvmos(cuda=True)

/home/lab-ws/miniconda3/envs/sula/lib/python3.9/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/home/lab-ws/miniconda3/envs/sula/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/lab-ws/miniconda3/envs/sula/lib/python3.9/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` A

In [22]:
mos= model.calculate_dir('new_six_denoised/', mean=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18024/18024 [04:16<00:00, 70.26it/s]


In [23]:
six_females['wvmos'] = mos

In [24]:
six_females.to_csv('six_mos.csv', index=False)

In [25]:
six_mos3 = six_females[six_females.wvmos>=3]

In [26]:
!mkdir 'six_mos3_wavs/wavs/'

In [27]:
for file_name in tqdm(six_mos3.path):
    source = 'new_six_denoised/' + file_name
    destination = 'six_mos3_wavs/wavs/' + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.move(source, destination)

  0%|          | 0/13132 [00:00<?, ?it/s]

## Text Preprocessing

In [28]:
import string
def clean_sentences(sentences):
    cleaned = []
    for sentence in sentences:
        cleaned_sentence = sentence.strip()
        cleaned.append(cleaned_sentence)
    return cleaned

six_mos3['cleaned_sentence'] = clean_sentences(six_mos3['sentence'])

/tmp/ipykernel_9797/1545359344.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  six_mos3['cleaned_sentence'] = clean_sentences(six_mos3['sentence'])


In [29]:
replacements = {
    '‘':'\'',
    '’':'\'',
    '“':'\'',
    '”':'\'',
    '"':'\'',
    '…':' ',
    '`':'',
}

def replace_characters(s, replacements):
    for c in replacements.keys():
        s = s.replace(c, replacements[c])
    return s

    # Use the translate() method to perform the replacements
#     return s.translate(translation_table)
six_mos3['replaced_sentence'] = six_mos3.cleaned_sentence.apply(lambda  x: replace_characters(x, replacements))

/tmp/ipykernel_9797/1773889350.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  six_mos3['replaced_sentence'] = six_mos3.cleaned_sentence.apply(lambda  x: replace_characters(x, replacements))


In [30]:
def generate_LJSpeech_line(df):
    lines = []
    for file, clean in zip(df.path, df.replaced_sentence):
        text = f'{file}|{clean}|{clean}'
        lines.append(text)
        with open('six_mos3_wavs/metadata.csv', 'w') as f:
            f.write(os.linesep.join(lines))
            
generate_LJSpeech_line(six_mos3)